In [1]:
import requests
import json
import datetime

In [2]:
# it is a good idea not to use exposed API key in program. Even better one is to salt it, but for local use this might be fine
# Store you API key inseparate txt file and give a path to it so ypu can retrieve it
X_API_KEY = open('/path/to/your/api/key.txt','r').readline()

In [3]:
# To get camera key first look at what cameras you have
cameras_response = requests.get("http://app.sigrow.com/api/v2/cameras", 
                                headers={"X-API-Key": X_API_KEY}).json()

In [4]:
# Select desired camera
selected_camera = cameras_response[0]['id']

In [5]:
# Select timeframe from what you would like to receive images for object labeling
# Bear in mind the time in the query should be in UTC! (See the API documentation)
after = datetime.datetime.strptime("13/09/2021 00:00:01", "%d/%m/%Y %H:%M:%S")
before = datetime.datetime.strptime("10/10/2021 23:59:59", "%d/%m/%Y %H:%M:%S")

# And get response with all available shots within given timeframe
shots_response = requests.get("http://app.sigrow.com/api/v2/camera/{}/shots?after={}&before={}".format(
                selected_camera, after.strftime("%Y%m%dT%H%M%SZ"), before.strftime("%Y%m%dT%H%M%SZ")
                ), headers={
                    "X-API-Key": X_API_KEY
                    }).json()

In [6]:
# Helper function to retrieve only RGB.jpg image
def get_rgb_source(item):
    return item["type"] == "RGB_JPG"

In [7]:
# Specify the path to the folder where you want your images to be stored
path='/path/to/folder/with/rgb/images/from/stomata_camera/'

In [ ]:
# Jet another way how to determine time from when you want your images to be retrieved
# Data in brackets corresponds to year, month, date, hour, minute,second
startdate=datetime.datetime(2021, 10, 10, 23, 59, 59, 0)

In [8]:
# Now lets iterate day by day - then you get data with 15 minute resolution
# in brackets this || numer (30) corresponds how many days back from date chosen you want to retrieve images
for day in range(0,30,1):
    
    # Here you are creating one day timestamp
    van=startdate - datetime.timedelta(days=(day+1))
    tot=startdate - datetime.timedelta(days=day)
    
    # Here you get all shots available at given day
    shots_response = requests.get("http://app.sigrow.com/api/v2/camera/{}/shots?after={}&before={}".format(
                        selected_camera, van.strftime("%Y%m%dT%H%M%SZ"), tot.strftime("%Y%m%dT%H%M%SZ")
                        ), headers={
                                    "X-API-Key": X_API_KEY
                                    }).json()
    
    # Now lets get RGB image from each shot - going one by one with increments by one
    for num in range(0,len(shots_response['shots']),1):

        # getting shot response to get shot ID
        selected_shot=shots_response["shots"][num]

        shot_source_response = requests.get("http://app.sigrow.com/api/v2/camera/{}/shot/{}/source".format(
            selected_camera, selected_shot["id"]
        ), headers={
            "X-API-Key": X_API_KEY
        }).json()
        rgb_source = list(filter(get_rgb_source, shot_source_response["sources"]))[0]

        # Getting image itself
        pic=requests.get(rgb_source["url"]).content

        # With some string splittin and rearranging we are getting original file name in Sigrow system
        fname=path+rgb_source["url"].split('/')[-1].split('.')[0]+"."+rgb_source["url"].split('/')[-1].split('.')[2]

        # Saving each image with its distinct filename to selected folder
        with open(fname, 'wb') as out_file:
            out_file.write(pic)

When you have necessary images, you can anotate your images to create dataset for object classification and recognition. 